In [1]:
#Becky Jacob
#CSC478 Homework 3

In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline
import os
from sklearn import feature_selection
from sklearn import cross_validation
import math

In [3]:
#################
#QUESTION 2
#################

In [4]:
#################
#Part a
#################
'''Create your own distance function that uses Cosine similarity. 
This is the distance function you will use to pass to the kMeans function.'''

'Create your own distance function that uses Cosine similarity. \nThis is the distance function you will use to pass to the kMeans function.'

In [5]:
from math import sqrt

In [6]:
def cosSim(vecA, vecB):
    sim = (np.dot(vecA,vecB))/((sqrt(np.dot(vecA,vecA))*(sqrt(np.dot(vecB,vecB)))))
    return 1-sim

In [7]:
#################
#Part b
#################
'''Load the data set. Matrix provided has terms as rows and documents as columns. 
Take the transpose of this matrix so that your main matrix is doc x term. 
Split the data set and set aside 20% for later use (see below). 
Use the 80% segment for clustering in the next part. 
The 20% portion must be a random subset.'''

'Load the data set. Matrix provided has terms as rows and documents as columns. \nTake the transpose of this matrix so that your main matrix is doc x term. \nSplit the data set and set aside 20% for later use (see below). \nUse the 80% segment for clustering in the next part. \nThe 20% portion must be a random subset.'

In [8]:
TxD = np.genfromtxt('matrix.txt', delimiter=',')
classes = np.genfromtxt('classes.txt', skip_header=1)

In [9]:
TxD.shape
#classes.shape

(9328, 2500)

In [10]:
classes[:20]

array([[  0.,   0.],
       [  1.,   1.],
       [  2.,   1.],
       [  3.,   1.],
       [  4.,   2.],
       [  5.,   1.],
       [  6.,   3.],
       [  7.,   4.],
       [  8.,   1.],
       [  9.,   1.],
       [ 10.,   2.],
       [ 11.,   4.],
       [ 12.,   1.],
       [ 13.,   0.],
       [ 14.,   0.],
       [ 15.,   2.],
       [ 16.,   0.],
       [ 17.,   0.],
       [ 18.,   3.],
       [ 19.,   4.]])

In [11]:
cla = classes[:,1]

In [12]:
cla

array([ 0.,  1.,  1., ...,  3.,  4.,  2.])

In [13]:
DxT = TxD.T
#ClassT = classes.T

In [14]:
DxT.shape
#ClassT.shape

(2500, 9328)

In [15]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(DxT, cla, test_size=0.2, random_state=35)

np.set_printoptions(precision=4, linewidth=80, suppress=True)
print (x_test.shape)
print (x_train.shape)
print (y_test.shape)
print (y_train.shape)

(500, 9328)
(2000, 9328)
(500,)
(2000,)


In [16]:
#################
#Part c
#################
'''Transform the term-frequencies to tfxidf values. 
Maintain DF values for each of the terms in the dictionary. 
[Note: if you run into problems due to limited computational resources, 
you may prune the data by removing all terms with low DF values, e.g., 
terms that appear in less than 10 documents. 
Be sure to maintain the correspondence between the dictionary terms and 
the matrix rows.]'''

'Transform the term-frequencies to tfxidf values. \nMaintain DF values for each of the terms in the dictionary. \n[Note: if you run into problems due to limited computational resources, \nyou may prune the data by removing all terms with low DF values, e.g., \nterms that appear in less than 10 documents. \nBe sure to maintain the correspondence between the dictionary terms and \nthe matrix rows.]'

In [17]:
x_train.shape[0]

2000

In [18]:
#TFxIDF weights
numTerms = x_train.shape[1]
numDocs = x_train.shape[0]
NMatrix = np.ones(np.shape(x_train), dtype=float)*numDocs
DF = np.array([(x_train!=0).sum(0)])
#DivM = np.divide(NMatrix, DF)
#IDF = np.log2(DivM)
IDF = np.log2(np.divide(NMatrix, DF))
pos_inf = float('+inf')
IDF[IDF == pos_inf] = 0.0
np.set_printoptions(precision=2)
print(DF)
print (IDF)
DT_tfidf = x_train*IDF

[[ 6  5 16 ...,  3  3  3]]
[[ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]
 [ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]
 [ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]
 ..., 
 [ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]
 [ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]
 [ 8.38  8.64  6.97 ...,  9.38  9.38  9.38]]


In [19]:
DT_tfidf.shape

(2000, 9328)

In [20]:
DT_tfidf

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [21]:
#Remove terms with DF<10 - I ended up not using this, because it wasn't taking too long to run


newM = np.vstack([DT_tfidf, DF])
newM.shape

reduced = newM[:,newM[2000]>10]
reduced.shape

reduced

final = reduced[:-1]

final.shape

In [22]:
#################
#Part d
#################
'''Perform Kmeans clustering on the training data.
Write a function to display the top N terms in each cluster along with 
the cluster DF values for each term and the size of the cluster. 
Cluster DF value for a term t in a cluster C is the percentage of docs 
in cluster C in which term t appears. Sort the terms in decreasing order 
of the DF percentage. Here is an example of how this output might look 
like (here the top 10 terms for 3 of the 5 clusters are displayed in 
decreasing order of cluster DF values, but the mean frequnecy from the 
cluster centroid is also shown). [Extra Credit: use your favorite third 
party tool, ideally with a Python based API, to create a word cloud for 
each cluster based on the in-cluster DF values.]'''

'Perform Kmeans clustering on the training data.\nWrite a function to display the top N terms in each cluster along with \nthe cluster DF values for each term and the size of the cluster. \nCluster DF value for a term t in a cluster C is the percentage of docs \nin cluster C in which term t appears. Sort the terms in decreasing order \nof the DF percentage. Here is an example of how this output might look \nlike (here the top 10 terms for 3 of the 5 clusters are displayed in \ndecreasing order of cluster DF values, but the mean frequnecy from the \ncluster centroid is also shown). [Extra Credit: use your favorite third \nparty tool, ideally with a Python based API, to create a word cloud for \neach cluster based on the in-cluster DF values.]'

In [23]:
from numpy import random

In [24]:
def randCent(dataSet, k):
        n = dataSet.shape[1]
        centroids = np.zeros((k,n), dtype=float)
        for j in range(n): #create random cluster centers
                minJ = min(dataSet[:,j])
                rangeJ = float(max(dataSet[:,j]) - minJ)
                centroids[:,j] = minJ + rangeJ * random.rand(k)
        return centroids 

In [25]:
def kMeans(dataSet, k, distMeas=cosSim, createCent=randCent):
    m = dataSet.shape[0]
    #print(m)
    clusterAssment = np.zeros((m,2))#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    #print (clusterAssment.shape)
    centroids = createCent(dataSet, k)
    #print (centroids)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = np.inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        #print (centroids)
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[np.nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = np.mean(ptsInClust, axis=0)
    return centroids, clusterAssment

In [26]:
centroids_tfidf, clusters_tfidf = kMeans(DT_tfidf, 3, cosSim, randCent)

In [27]:
terms = np.genfromtxt('terms.txt',dtype=str)

In [28]:
centroids_tfidf

array([[ 0.06,  0.  ,  0.32, ...,  0.06,  0.04,  0.08],
       [ 0.  ,  0.  ,  0.  , ...,  1.04,  0.  ,  0.  ],
       [ 0.  ,  0.1 ,  0.02, ...,  0.  ,  0.  ,  0.  ]])

In [29]:
clusters_tfidf

array([[ 0.  ,  0.77],
       [ 0.  ,  0.84],
       [ 2.  ,  0.85],
       ..., 
       [ 2.  ,  0.49],
       [ 0.  ,  0.86],
       [ 2.  ,  0.66]])

In [39]:
def TopNTerms(N, k, data, terms):
    centroids_tfidf, clusters_tfidf = kMeans(data, k, cosSim, randCent)
    clus = clusters_tfidf[:,0]
    pdDF = pd.DataFrame(data)
    pdDF.columns = [terms[i] for i in range(len(terms))]
    #print (pdDF)
    pdDF['predClass'] = pd.Series(clus)
    for i in range(0,k):
        clusData = pdDF.loc[pdDF['predClass']==i]
        NumDocs = clusData.astype(bool).sum(axis=0).tolist()
        temp = [(x/len(clusData))*100 for x in NumDocs]
        PerDoc = [ '%.4f' % d for d in temp]
        lstTemp = pd.DataFrame(list(zip(NumDocs,PerDoc)))
        lstTemp.columns = ['Frequency', 'Percentage']
        lstTemp.insert(loc=0,column='Words',value=pd.Series(terms))
        lstFinal = lstTemp.sort_values(['Percentage'],ascending=False)
        print ('Cluster: {0}  results'.format(i))
        print ('Number of Documents in Cluster: {0}'.format(len(clusData)))
        print (lstFinal.head(N))

In [40]:
TopNTerms(5,5,x_train, terms)

Cluster: 0  results
Number of Documents in Cluster: 1986
       Words  Frequency Percentage
5975    part        197     9.9194
3419   great        197     9.9194
2174  differ        196     9.8691
3239   gener        196     9.8691
732   better        195     9.8187
Cluster: 1  results
Number of Documents in Cluster: 1
        Words  Frequency Percentage
9328      NaN          1   100.0000
5758   offset          1   100.0000
8470      two          1   100.0000
878   borland          1   100.0000
895     bounc          1   100.0000
Cluster: 2  results
Number of Documents in Cluster: 7
     Words  Frequency Percentage
5255  mnui          6    85.7143
798    biz          6    85.7143
4835    ma          6    85.7143
9307   zkn          6    85.7143
9306   zkm          6    85.7143
Cluster: 3  results
Number of Documents in Cluster: 6
     Words  Frequency Percentage
5826    oq          5    83.3333
4725    lk          5    83.3333
3482    gz          5    83.3333
7664    sp          5    

In [41]:
#################
#Part e
#################
'''Using the cluster assignments from Kmeans clustering, 
compare your 5 clusters to the 5 pre-assigned classes by computing 
the Completeness and Homogeneity values.'''

'Using the cluster assignments from Kmeans clustering, \ncompare your 5 clusters to the 5 pre-assigned classes by computing \nthe Completeness and Homogeneity values.'

In [42]:
newC = np.ravel(clusters_tfidf.T[0])
newC = newC.astype(int)
print (newC)

[0 0 2 ..., 2 0 2]


In [43]:
from sklearn.metrics import completeness_score, homogeneity_score

In [44]:
print (completeness_score(y_train,newC))

0.815860793579


In [45]:
print (homogeneity_score(y_train,newC))

0.275284108844
